## Setup

1. Complete the previous challenge.
2. Create a Cognitive Services resource.

In [1]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICE_KEY"

## Function to wrap HTTP requests

In [2]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2017-11-11-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


## Verify Data Source Exists

In [3]:
import json

try:
    result = azsearch_rest(function_name="datasources")
    if len(result) > 0:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

200
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#datasources",
  "value": [
    {
      "@odata.etag": "\"0x8D6C45C6B278563\"",
      "container": {
        "name": "margies-data",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": "Various documents.",
      "name": "website-docs",
      "subtype": null,
      "type": "azureblob"
    }
  ]
}


## Update the Index

In [4]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The sentiment score
      "name": "sentiment",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": True
    },
    {
      # Key phrases that might help identify the subject of the text
      "name": "key_phrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False
    },
    {
      # Geographical locations mentioned in the text
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # People mentioned in the text
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "links",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },    {
      # The image content in the document
      "name": "image_descriptions",
      "type": "Collection(Edm.String)",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The text contained in images in the document
      "name": "image_text",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The combined source text plus the text extracted from images
      "name": "merged_text",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": False,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Define a Skillset

In [5]:
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
      # Extract text from images (including those embedded in documents)
        {
            "description": "Extract descriptions from images.",
            "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "visualFeatures": ["Description"],
            "details": ["Landmarks"],
            "inputs": [
              {
                "name": "image",
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "description",
                "targetName": "imageDescription"
              }
            ]
        },
      # Extract text from images (including those embedded in documents)
        {
            "description": "Extract text from images.",
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
              {
                "name": "image",
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "text"
              }
            ]
        },
       # Merge the extracted image text back into the content at the appropriate locations
        {
          "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
          "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
          "context": "/document",
          "insertPreTag": "[",
          "insertPostTag": "]",
          "inputs": [
            {
              "name":"text", "source": "/document/content"
            },
            {
              "name": "itemsToInsert", "source": "/document/normalized_images/*/text"
            },
            {
              "name":"offsets", "source": "/document/normalized_images/*/contentOffset" 
            }
          ],
          "outputs": [
            {
              "name": "mergedText"
            }
          ]
        },
         # Detect the sentiment
        {
          "description": "Detect sentiment.",
          "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
        "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
            "outputs": [
                {
                  "name": "score",
                  "targetName": "sentimentScore"
                }
            ]
        },
       # Extract entities
        {
          "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
          "categories": [ "Organization", "location", "person", "datetime", "url" ],
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "locations"
            },
            {
              "name": "persons"
            },
            {
              "name": "urls"
            },
            {
              "name": "entities"
            }
          ]
        },
      # Extract key phrases from the pages
      {
          "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
          "context": "/document",
          "defaultLanguageCode": "en",
          "maxKeyPhraseCount": 5,
          "inputs": [
            {
              "name": "text", "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "keyPhrases"
            }
          ]
        },  
      ],
      "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
      }
    }  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/website-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


## Create an Indexer

In [6]:
import json

body = {
  "name":"website-docs-indexer", 
  "dataSourceName" : "website-docs",
  "skillsetName" : "website-skillset",
  "targetIndexName" : "website-docs-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from cognitive skills to index fields
        {
            "sourceFieldName": "/document/normalized_images/*/imageDescription", 
            "targetFieldName": "image_descriptions"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/text",
            "targetFieldName": "image_text"
        },
        {
            "sourceFieldName": "/document/mergedText", 
            "targetFieldName": "merged_text"
        },
        {
            "sourceFieldName": "/document/sentimentScore", 
            "targetFieldName": "sentiment"
        },
        {
          "sourceFieldName" : "/document/persons", 
          "targetFieldName" : "people"
        },
        {
          "sourceFieldName" : "/document/locations", 
          "targetFieldName" : "locations"
        },
        {
          "sourceFieldName" : "/document/urls", 
          "targetFieldName" : "links"
        },
        {
          "sourceFieldName" : "/document/entities", 
          "targetFieldName" : "entities"
        },
        {
          "sourceFieldName" : "/document/keyPhrases/*", 
          "targetFieldName" : "key_phrases"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata",
        "imageAction": "generateNormalizedImages"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/website-docs-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [7]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [8]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
reset
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Create Wrapper Function for Query Requests

In [2]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### Collateral Images

In [3]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, image_descriptions, image_text',
    '$filter': "search.ismatch('collateral', 'url') and image_descriptions/any()",
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        for i in range(len(value["image_descriptions"])):
            desc_json = json.loads(value["image_descriptions"][i])
            print("\tImage:",desc_json["captions"][0]["text"], ", Text:", value["image_text"][i])

except Exception as e:
    print('Error:')
    print(e)

Search: The Grand Canyon
Hits: 1

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Las%20Vegas%20Brochure.pdf
Las Vegas Brochure.pdf
	Image: a view of a city at night , Text: The Strip at night
	Image: a canyon with a mountain in the background , Text: The Grand Canyon - a short daytrip from Las Vegas


### Search in Image Descriptions

In [10]:
import urllib.parse, json

search_terms = input("Find documents containing images of: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'image_descriptions:"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, image_descriptions',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"])
        for img_description in value["image_descriptions"]:
            desc_json = json.loads(img_description)
            for caption in desc_json["captions"]:
                print("Image:",caption["text"])
                print("Tags:")
                for tag in desc_json["tags"]:
                    print("\t", tag)

except Exception as e:
    print('Error:')
    print(e)

Find documents containing images of: bridge
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/San%20Francisco%20Brochure.pdf
San Francisco Brochure.pdf
Image: a train crossing Golden Gate Bridge over a river
Tags:
	 outdoor
	 bridge
	 train
	 building
	 crossing
	 water
	 track
	 river
	 traveling
	 view
	 going
	 large
	 seen
	 long
	 blue
	 crane
	 city
	 white
	 red
	 tall
	 flying
	 riding
Image: a large tall tower with a clock on the side of a building
Tags:
	 outdoor
	 building
	 tall
	 skyscraper
	 tower
	 clock
	 large
	 background
	 high
	 city
	 side
	 clear
	 big
	 blue
	 green
	 light
	 air
	 red
	 flying
	 traffic
	 white

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/London%20Brochure.pdf
London Brochure.pdf
Image: a large bridge over some water
Tags:
	 outdoor
	 water
	 building
	 bridge
	 large
	 river
	 sitting
	 boat
	 front
	 cloudy
	 clock
	 train
	 green
	 traveling
	 old
	 standing
	 ci